In [8]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import *

In [9]:
cancer = pd.read_csv(r"/home/sarthakredasani/Documents/CDAC_ML/Cases/Cases/Cancer/Cancer.csv", index_col =0)
cancer.head(5)

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
subjid,,,,,,,,,,
1,40-49,premeno,15 to 19,0 to 2,yes,three,right,left_up,no,recurrence-events
2,50-59,ge40,15 to 19,0 to 2,no,one,right,central,no,no-recurrence-events
3,50-59,ge40,35 to 39,0 to 2,no,two,left,left_low,no,recurrence-events
4,40-49,premeno,35 to 39,0 to 2,yes,three,right,left_low,yes,no-recurrence-events
5,40-49,premeno,30 to 34,3 to 5,yes,two,left,right_up,no,recurrence-events


In [10]:
X, y = cancer.drop('Class', axis =1), cancer['Class']

In [11]:
le = LabelEncoder()
le_y = le.fit_transform(y)

In [12]:
X_train,X_test, y_train, y_test = train_test_split(X, le_y, test_size=0.3, random_state=25,
                                                  stratify=le_y)

In [13]:
ohe = OneHotEncoder(sparse_output=False, drop="first").set_output(transform='pandas')
X_trn_ohe = ohe.fit_transform(X_train)
X_tst_ohe = ohe.transform(X_test)
X_trn_ohe.shape

(200, 34)

In [14]:
nb = BernoulliNB()
nb.fit(X_trn_ohe, y_train)
y_pred = nb.predict(X_tst_ohe)
print(accuracy_score(y_test, y_pred))

0.7441860465116279


In [15]:
y_pred_prob = nb.predict_proba(X_tst_ohe)
roc_auc_score(y_test, y_pred_prob[:,1])

0.7653846153846154

In [16]:
alphas = np.linspace(0.001, 5, 20)
scores = []
for a in alphas:
    nb = BernoulliNB(alpha=a)
    nb.fit(X_trn_ohe, y_train)
    y_pred = nb.predict(X_tst_ohe)
    y_pred_prob = nb.predict_proba(X_tst_ohe)
    scores.append([a, accuracy_score(y_test, y_pred), roc_auc_score(y_test, y_pred_prob[:,1])])
df_scores = pd.DataFrame(scores, columns=['alpha','acc','auc'])
df_scores.sort_values('auc', ascending=False).iloc[:3]

,alpha,acc,auc
3,0.790316,0.744186,0.767308
2,0.527211,0.744186,0.766667
1,0.264105,0.732558,0.766667
